In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import svm
from sklearn import metrics
from sklearn import tree

In [2]:
df = pd.read_csv("C:\\Users\\Raaed\\Downloads\\Emotions Edited.csv") ## Loading the dataset

## Data Preprocessing 

In [3]:
df

,face,confidence,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,Target,Emotion
0,0,0.975,0.172135,0.143987,-0.974493,-0.118361,0.165401,-0.979098,0.028,0.157,...,1,1,0,1,0,0,0,1,0,Anger
1,0,0.975,0.074368,0.131744,-0.988490,-0.103927,0.197212,-0.974837,-0.015,0.166,...,1,1,0,1,0,0,0,0,0,Anger
2,0,0.975,0.085587,0.098758,-0.991424,-0.141882,0.136020,-0.980494,-0.029,0.119,...,0,1,1,0,0,1,0,1,0,Anger
3,0,0.975,0.157062,0.207493,-0.965545,-0.110608,0.218821,-0.969476,0.024,0.217,...,1,1,1,0,0,1,0,0,0,Anger
4,0,0.975,0.121276,0.246677,-0.961479,-0.167919,0.242985,-0.955386,-0.024,0.250,...,0,0,1,0,0,0,0,0,0,Anger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,0,0.875,0.208317,0.085872,-0.974284,-0.066278,0.041319,-0.996945,0.072,0.064,...,0,1,1,0,1,1,0,0,1,Surprise
322,0,0.975,0.070607,0.146165,-0.986737,-0.067520,0.185956,-0.980235,0.002,0.167,...,1,0,0,0,1,1,0,1,1,Surprise
323,0,0.925,0.074448,0.162938,-0.983824,-0.083946,0.108327,-0.990565,-0.005,0.137,...,0,0,0,1,1,1,0,0,1,Surprise
324,0,0.975,0.179769,0.211315,-0.960744,-0.045008,0.218831,-0.974724,0.070,0.219,...,0,0,0,0,1,1,0,0,1,Surprise


In [4]:
df.drop('Emotion',axis = 1, inplace = True)

In [5]:
col = df.columns[0:676]
df.drop(col,inplace = True, axis =1 )

In [6]:
## Dividing the data into training, validation and testing sets 
x = df.drop('Target',axis =1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.10,random_state =42) 
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size = 0.20,random_state = 42)

## Particle Swarm Optimization

In [7]:
#Initializing the hyperparameters
w=0.8
c1=2
c2=2
popsize=30
itr=500
gbestfitness= 0
gbest=np.random.uniform(0,0,len(x.columns))

In [8]:
class Particle:
    def __init__(self):
        self.pos=np.random.uniform(0,1,len(x.columns)) ## initializing a random start position
        self.vel=np.random.uniform(0,1,len(x.columns)) ## initializing random velocity
    
        self.bits=np.random.uniform(0,0,len(x.columns))
        for i in range(0,len(x.columns)):
            self.bits[i] = self.pos[i]
            if self.bits[i]>0.94:
                self.bits[i] = 1 
            else:
                self.bits[i] = 0
        self.bits = self.bits.astype(int)
    
        self.fitness = 0 
        self.lbfitness = 0
        self.lbest = self.pos
    

In [9]:
## constructing the particle swarm 
particles = [] 
for i in range(popsize):
    particles.append(Particle())

## PSO Algorithm 

In [10]:
gbest_accuracy = [] 
for j in range(0,itr):
    for i in range(0,popsize):
        lst = [] 
        for k in range(0,len(x.columns)):
            if particles[i].bits[k]==0:
                lst.append(k)
        xPSO = X_train.drop(X_train.columns[lst],axis=1) ## selected feature set
        
        clf = svm.SVC()## linear kernel     
        ##training the model
        try:
            clf.fit(xPSO,y_train)
        
        #predict the responses for test dataset
            xPSO_val = X_val.drop(X_val.columns[lst],axis=1)
        
            y_pred = clf.predict(xPSO_val)
            particles[i].fitness = metrics.accuracy_score(y_val,y_pred)
        except:
            particles[i].fitness = 0
        
        if particles[i].fitness>particles[i].lbfitness: 
            particles[i].lbfitness = particles[i].fitness
            particles[i].lbest = particles[i].pos
        
        if particles[i].lbfitness > gbestfitness:
            gbestfitness = particles[i].lbfitness
            gbest = particles[i].lbest
    
    gbest_accuracy.append(gbestfitness)
    
    for i in range(0,popsize): 
        particles[i].vel = w*particles[i].vel + c1*random.random()*(particles[i].lbest - particles[i].pos)+ c2*random.random()*(gbest - particles[i].pos)
        particles[i].pos = particles[i].pos + particles[i].vel 
        

In [11]:
gbbits = np.random.uniform(0,0,len(x.columns))

for i in range(0,len(x.columns)):
    gbbits[i] = gbest[i]
    if gbbits[i] > 0.94: 
        gbbits[i] = 1
    else: 
        gbbits[i] = 0 
gbbits = gbbits.astype(int)

lst1=[]
for i in range(0,len(x.columns)):
    if gbbits[i]==0:
        lst1.append(i)
        
gbxPSO = X_train.drop(X_train.columns[lst1],axis=1) 

In [12]:
sum(gbbits) ## the number of features selected 

1

## Testing 

In [13]:
clf =  svm.SVC(kernel = 'linear') # Linear Kernel
##clf = tree.DecisionTreeClassifier()
clf.fit(gbxPSO,y_train)

gbxPSO_test = X_test.drop(X_test.columns[lst1],axis=1) 
y_pred = clf.predict(gbxPSO_test)
accuracy = metrics.accuracy_score(y_test,y_pred)
print(100*accuracy)

90.9090909090909
